# Notebook 7b: 

simple plot ipsi contra combined win stay and lose switch conditions. recordings from the right side hemisphere. 

In [1]:
import os
from pathlib import Path
import pickle
import matplotlib.pyplot as plt

In [2]:
import Notebook_7_helper_functions as Nb_7

In [ ]:
Nb_name= 'Notebook_7_b'
period_of_interest = "combined_periods_1"

In [ ]:
mouse = 'T240'
data_day = '2019_09_27__T240'
date = '2019_09_27'

HowManyBack = 1

seq_str= "0a1b2a3b4abcde5a6a7b"

data_dir_output = "/Users/gilmandelbaum/Desktop/outPut"

#corresponds to the number 3 notebooks: 
y_axis_list = ["df_f","z_score","z_score_sliding_window"]

In [ ]:
# Parameters
data_dir_input = (
    "/Volumes/GilInVivoPaper.large/2.Photometry_in_str/0.data_Sets/1.lateral_medial_6/"
)
data_dir_output = (
    "/Volumes/GilInVivoPaper.large/2.Photometry_in_str/1.lateral_medial_6_analysis/"
)
drop2blocks = "yes"
drop2lastblocks = "yes"
seq_str = "0a1b2b3b4abcd5a6a7b"
HowManyBack = 1
Nb0 = "a"
Nb1 = "b"
Nb2 = "b"
Nb3 = "b"
Nb4 = ["a", "b", "c", "d"]
Nb5 = "a"
Nb6 = "a"
Nb7 = "b"
Tags = [
    "Reward_NoReward_tag",
    "ENL_tag",
    "Cue_to_Sel_tag",
    "Cue_to_Sel_fromPrevTrial_tag",
]
Timing = [600, 200, 15, 15]
Start_or_end = ["start", "end", "start", "start"]
y_axis_list = ["df_f", "z_score", "z_score_sliding_window"]
mouse = "T163"
data_day = "2018_11_27__T163"
date = "2018_11_27"
photo_day = "T163-144959"


same for all 7 notebooks

In [ ]:
alphabet = ["a","b","c","d","e","f","g"]
y_axis = y_axis_list[alphabet.index(seq_str[:seq_str.index('4')][-1])]
print (y_axis)

In [ ]:
version_to_import = seq_str[:seq_str.index('7')][-1]

In [ ]:
root = Path(data_dir_output+"/"+mouse+"/"+data_day+'/'+str(HowManyBack)+"_Back")
d = mouse+"_"+date+"Notebook_6_"+version_to_import+"_"+'seq'+seq_str[:seq_str.index('7')]+'.pickle'

my_path = root / d 
print (my_path)
        
# open a file, where you stored the pickled data
fileToOpen = open(my_path, 'rb')
# load the pickle: 
PhotoData_perTrial_channels = pickle.load(fileToOpen)      

In [ ]:
# stash result in a cache directory, for how many back. 
cache_dir_plt = os.path.join(data_dir_output+'/'+mouse+'/'+data_day, str(HowManyBack)+'_Back',"0.plots","session_plots_"+y_axis)
if not os.path.exists(cache_dir_plt):
    os.makedirs(cache_dir_plt)    

In [ ]:
path_to_plot = cache_dir_plt+"/"+seq_str
print (path_to_plot)

# extract data of interest_right_side

In [ ]:
r_ls_combined_periods_1_d1= Nb_7.extract_data_of_interest_ipsi_contra_per_session(PhotoData_perTrial_channels,
                                                                      0,0,"combined_periods_1","d1 R")

In [ ]:
r_ls_combined_periods_1_d2= Nb_7.extract_data_of_interest_ipsi_contra_per_session(PhotoData_perTrial_channels,
                                                                      0,0,"combined_periods_1","d2 R")

In [ ]:
r_wr_combined_periods_1_d1= Nb_7.extract_data_of_interest_ipsi_contra_per_session(PhotoData_perTrial_channels,
                                                                      0,3,"combined_periods_1","d1 R")

In [ ]:
r_wr_combined_periods_1_d2= Nb_7.extract_data_of_interest_ipsi_contra_per_session(PhotoData_perTrial_channels,
                                                                      0,3,"combined_periods_1","d2 R")

# calculate_mean_sem per session_right_side

In [ ]:
r_ls_combined_periods_1_d1_mean_sem = Nb_7.calculate_mean_sem (r_ls_combined_periods_1_d1)

In [ ]:
r_ls_combined_periods_1_d2_mean_sem = Nb_7.calculate_mean_sem(r_ls_combined_periods_1_d2)

In [ ]:
r_wr_combined_periods_1_d1_mean_sem = Nb_7.calculate_mean_sem(r_wr_combined_periods_1_d1)

In [ ]:
r_wr_combined_periods_1_d2_mean_sem = Nb_7.calculate_mean_sem(r_wr_combined_periods_1_d2)

# make_sem_traces_right_side

In [ ]:
r_ls_combined_periods_1_d1_sem_traces = Nb_7.make_sem_traces(r_ls_combined_periods_1_d1_mean_sem)

In [ ]:
r_ls_combined_periods_1_d2_sem_traces = Nb_7.make_sem_traces(r_ls_combined_periods_1_d2_mean_sem)

In [ ]:
r_wr_combined_periods_1_d1_sem_traces = Nb_7.make_sem_traces(r_wr_combined_periods_1_d1_mean_sem)

In [ ]:
r_wr_combined_periods_1_d2_sem_traces = Nb_7.make_sem_traces(r_wr_combined_periods_1_d2_mean_sem)

# plot_right_side 

In [ ]:
Nb_7.plot_ipsi_contra_together(r_ls_combined_periods_1_d1_mean_sem,
                              r_ls_combined_periods_1_d1_sem_traces,
                              "right_side_lose switch","D1",y_axis,path_to_plot)

In [ ]:
Nb_7.plot_ipsi_contra_together(r_ls_combined_periods_1_d2_mean_sem,
                              r_ls_combined_periods_1_d2_sem_traces,
                              "right_side_lose switch","D2",y_axis,path_to_plot)

In [ ]:
Nb_7.plot_ipsi_contra_together(r_wr_combined_periods_1_d1_mean_sem,
                              r_wr_combined_periods_1_d1_sem_traces,
                              "right_side_win repeat","D1",y_axis,path_to_plot)

In [ ]:
Nb_7.plot_ipsi_contra_together(r_wr_combined_periods_1_d2_mean_sem,
                              r_wr_combined_periods_1_d2_sem_traces,
                               "right_side_win repeat","D2",y_axis,path_to_plot)

In [ ]:
root = Path(data_dir_output+"/"+mouse+"/"+data_day+'/'+str(HowManyBack)+"_Back")
d = mouse+"_"+date+Nb_name+'_seq'+seq_str+'.pickle'
my_path = root / d 
my_file = open(my_path, 'wb')
my_file = pickle.dump((),my_file)